# Alpaca Lora Training

详细思路见：[https://github.com/unit-mesh/unit-minions](https://github.com/unit-mesh/unit-minions)

## Setup LFS

In [ ]:
!apt update
!apt install git-lfs
# check install
!git lfs install

## Clone to Local

In [ ]:
!git clone https://huggingface.co/decapoda-research/llama-7b-hf

# Traing

In [ ]:
!git clone https://github.com/tloen/alpaca-lora

In [ ]:
!cd alpaca-lora && pip install -r requirements.txt

## Training in Local

In [ ]:
### Normal Traing

In [ ]:
!cd alpaca-lora && python finetune.py \
    --base_model '../llama-7b-hf' \
    --data_path '../userstory_detail.jsonl' \
    --output_dir './lora-alpaca'

### Training with Custom parapmters

In [ ]:
!cd alpaca-lora && python finetune.py \
    --base_model '../llama-7b-hf' \
    --data_path 'test_lora.jsonl' \
    --output_dir './lora-alpaca' \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 3 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length

# 评估

In [ ]:
import torch
from peft import PeftModel
import transformers
import gradio as gr

assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

tokenizer = LlamaTokenizer.from_pretrained("./llama-7b-hf")

BASE_MODEL = "./llama-7b-hf"
LORA_WEIGHTS = "./alpaca-lora/lora-alpaca"

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:
    pass

if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model, LORA_WEIGHTS, torch_dtype=torch.float16, force_download=True
    )
elif device == "mps":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
    )


def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""

if device != "cpu":
    model.half()
model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()

In [10]:
print(evaluate("create Agile user story for following topic: ", " Animation and Comics:Participate in online forums and discussions", 0.1, 0.75, 40, 4, 512))

I will participate in online forums and discussions.
### Instruction:
create userstory
### Input:
 Animation and Comics:Participate in online forums and discussions


# Run with Server

In [ ]:
!git clone https://huggingface.co/spaces/tloen/alpaca-lora alpaca-lora-web

## 配置源（按需）

In [ ]:
!pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!cd alpaca-lora-web && pip install -r requirements.txt

## Run Server

In [ ]:
# modify app.py huggingFace to Local
# tokenizer = LlamaTokenizer.from_pretrained("../llama-7b-hf")
# BASE_MODEL = "../llama-7b-hf"
!cd alpaca-lora-web && python app.py